# Challenge Kaggle: How Much Did It Rain? II
  - Etudiant: Ghilas BELHADJ
  - Challenge URL: https://www.kaggle.com/c/how-much-did-it-rain-ii
  - /!\ IPython NoteBook ne semble pas répondre lorsque je lance les algorithmes d'aprentissage dessus, les algo ont été lancé sur une console. 

## Description
Il s'agit dans ce challenge de prédire la quantité d'eau de pluie (en mm) recueillit dans des jauges prevu à cet effet sur un interval de temps d'une heure. Pour celà nous disposons de données concernant des précipitations antérieurs. Nous allons dans ce qui suit, essayer d'apprendre un modèle qui puisse faire cette prédiction là.

## Outils

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime as dt
import cPickle as cpk

import sklearn
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn import cross_validation
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error

## Données du challenge
[Les données du challenge](https://www.kaggle.com/c/how-much-did-it-rain-ii/data) doivent se trouver dans le dossier `data` afin de pouvoir executer ce IPython Noteook.
```
data
   ├── test.csv
   └── train.csv
```

Les données sont sous formes de séquences groupées par heure, les entrées qui appartiennent à la même heure ont un identifiant commun et unique. Ensuite chaque entrées comprend differentes mesures prise par un radar à un instant donné.


## Description des attributs
On vas se reférer à la [description donnée sur Kaggle](https://www.kaggle.com/c/how-much-did-it-rain-ii/data) pour voir à quoi correspond chaque attribut.

  * Id: Un identifiant unique d'un ensemble d'observations sur une heure
  * minutes_past: La minute à laquelle l'observation a été faite ( relative a l'heure d'observation de chaque ensemble d'observations )
  * radardist_km: Dsitance de la jauge par rapport au radar qui prend la mesure sur cette gauge.
  * Ref: [Réflectivité](https://fr.wikipedia.org/wiki/R%C3%A9flectivit%C3%A9) en km ( c'est un rapport entre l'énergie réfléchie par rapport à l'énergie incidente d'un objet)
  * Ref_5x5_10th: 10émè [centile](https://fr.wikipedia.org/wiki/Centile) de reflexivité dans un voisinage 5x5 autour de la gauge.
  * Ref_5x5_50th: 50ème centile.
  * Ref_5x5_90th: 90ème centile.
  * RefComposite: Reflexivité maximum sur la colonne vertical au dessus de la gauge ( en [décibel Z](https://fr.wikipedia.org/wiki/D%C3%A9cibel_Z) )
  * RefComposite_5x5_10th: 10ème centile.
  * RefComposite_5x5_50th: 50ème centile.
  * RefComposite_5x5_90th: 90ème centile.
  * RhoHV: Un coeficient de correlation (sans unité)
  * RhoHV_5x5_10th: 10ème centile.
  * RhoHV_5x5_50th: 50ème centile.
  * RhoHV_5x5_90th: 90ème centile.
  * Zdr: Reflectivité différentiel en [Décibel](https://fr.wikipedia.org/wiki/D%C3%A9cibel)
  * Zdr_5x5_10th: 10ème centile.
  * Zdr_5x5_50th: 50ème centile.
  * Zdr_5x5_90th: 90ème centile.
  * Kdp: La phase différentielle spécifique (Utilisé pour localiser les régions à forte précipitations/Attenuations)
  * Kdp_5x5_10th: 10ème centile.
  * Kdp_5x5_50th: 50ème centile.
  * Kdp_5x5_90th: 90ème centile.
  * Expected: L'observation actuelle de l'etat de la gauge (en mm)


## Aperçu des données
#### Lecture des données

In [3]:
data_path = "data/"
train_file= "train.csv"
test_file= "test.csv"

print "Reading %s" % train_file
time0 = dt.datetime.now()

df_train = pd.read_csv(data_path+train_file)
df_train.set_index('Id')

# test_file = pd.read_csv(data_path+train_file)
# test_file.set_index('Id')

time1 = dt.datetime.now()
print('Read in %i sec' % (time1-time0).seconds)

Reading train.csv
Read in 30 sec


## Analyse des données

In [4]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13765201 entries, 0 to 13765200
Data columns (total 24 columns):
Id                       int64
minutes_past             int64
radardist_km             float64
Ref                      float64
Ref_5x5_10th             float64
Ref_5x5_50th             float64
Ref_5x5_90th             float64
RefComposite             float64
RefComposite_5x5_10th    float64
RefComposite_5x5_50th    float64
RefComposite_5x5_90th    float64
RhoHV                    float64
RhoHV_5x5_10th           float64
RhoHV_5x5_50th           float64
RhoHV_5x5_90th           float64
Zdr                      float64
Zdr_5x5_10th             float64
Zdr_5x5_50th             float64
Zdr_5x5_90th             float64
Kdp                      float64
Kdp_5x5_10th             float64
Kdp_5x5_50th             float64
Kdp_5x5_90th             float64
Expected                 float64
dtypes: float64(22), int64(2)
memory usage: 2.6 GB


Les attributs sont tous de type numérique, Id et minutes_past sont des valeurs discrètes, et les autres sont des valeurs continues   

In [5]:
df_train.head(10)

,Id,minutes_past,radardist_km,Ref,Ref_5x5_10th,Ref_5x5_50th,Ref_5x5_90th,RefComposite,RefComposite_5x5_10th,RefComposite_5x5_50th,...,RhoHV_5x5_90th,Zdr,Zdr_5x5_10th,Zdr_5x5_50th,Zdr_5x5_90th,Kdp,Kdp_5x5_10th,Kdp_5x5_50th,Kdp_5x5_90th,Expected
0,1,3,10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.254
1,1,16,10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.254
2,1,25,10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.254
3,1,35,10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.254
4,1,45,10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.254
5,1,55,10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.254
6,2,1,2,9.0,5.0,7.5,10.5,15.0,10.5,16.5,...,0.998333,0.3750,-0.1250,0.3125,0.8750,1.059998,-1.410004,-0.350006,1.059998,1.016
7,2,6,2,26.5,22.5,25.5,31.5,26.5,26.5,28.5,...,1.005000,0.0625,-0.1875,0.2500,0.6875,NaN,NaN,NaN,1.409988,1.016
8,2,11,2,21.5,15.5,20.5,25.0,26.5,23.5,25.0,...,1.001667,0.3125,-0.0625,0.3125,0.6250,0.349991,NaN,-0.350006,1.759994,1.016
9,2,16,2,18.0,14.0,17.5,21.0,20.5,18.0,20.5,...,1.001667,0.2500,0.1250,0.3750,0.6875,0.349991,-1.059998,0.000000,1.059998,1.016


Juste avec les 10 premières ligne du fichier train.csv, on remarque qu'il y a ennormement de valeurs manquantes (NaN). nous allons continuer a explorer nos données pour voir si celà est pareil sur l'ensemble du jeu de données.

In [6]:
df_train.describe()
# df_test.describe()

,Id,minutes_past,radardist_km,Ref,Ref_5x5_10th,Ref_5x5_50th,Ref_5x5_90th,RefComposite,RefComposite_5x5_10th,RefComposite_5x5_50th,...,RhoHV_5x5_90th,Zdr,Zdr_5x5_10th,Zdr_5x5_50th,Zdr_5x5_90th,Kdp,Kdp_5x5_10th,Kdp_5x5_50th,Kdp_5x5_90th,Expected
count,13765201.000000,13765201.000000,13765201.000000,6349375.000000,5283988.000000,6356482.000000,7551281.000000,6716343.000000,5755673.000000,6711663.000000,...,5905584.000000,4934916.000000,4133154.000000,4936568.000000,5905584.000000,4182635.000000,3428782.000000,4187281.000000,5052776.000000,13765201.000000
mean,592336.986614,29.523733,11.067943,22.926658,19.952271,22.610287,25.898461,24.711081,22.158238,24.420753,...,1.015272,0.536709,-0.719008,0.337622,2.072870,0.035452,-3.482325,-0.473655,4.079836,108.626306
std,340856.086251,17.308131,4.206618,10.355157,9.208166,10.053000,11.109579,10.689622,9.702705,10.424526,...,0.048616,1.510399,1.006068,0.938644,1.670194,3.869725,2.792120,2.263046,4.147337,548.605805
min,1.000000,0.000000,0.000000,-31.000000,-32.000000,-32.000000,-28.500000,-32.000000,-31.000000,-27.500000,...,0.208333,-7.875000,-7.875000,-7.875000,-7.875000,-96.040000,-80.790000,-78.770004,-100.200005,0.010000
25%,296897.000000,15.000000,9.000000,16.000000,14.000000,16.000000,18.000000,17.500000,16.000000,17.500000,...,0.998333,-0.187500,-1.125000,-0.062500,1.062500,-1.410004,-4.580002,-0.710007,2.069992,0.254000
50%,592199.000000,30.000000,11.000000,22.500000,20.000000,22.500000,25.500000,24.000000,22.000000,24.000000,...,1.011667,0.375000,-0.625000,0.250000,1.687500,0.000000,-2.820007,0.000000,3.519989,1.016000
75%,889582.000000,44.000000,14.000000,29.500000,26.000000,29.000000,33.500000,31.500000,28.500000,31.500000,...,1.051667,1.062500,-0.187500,0.687500,2.625000,1.750000,-1.760010,0.349991,5.639999,3.810002
max,1180945.000000,59.000000,21.000000,71.000000,62.500000,69.000000,72.500000,92.500000,66.000000,71.000000,...,1.051667,7.937500,7.937500,7.937500,7.937500,179.750000,3.519989,12.800003,144.600000,33017.730000


* On remarque que sur 13 765 201 lignes, ~7 000 000 valeurs sont NaN sur chaque attribue, soit plus de 50% sur chaque colonne.

* On remarque aussi que des eventuels outliers se sont introduit dans la colonne Expected, on peu remarquer ça en observant la difference entre les quartils des données et la valeur max. On peu aussi confirmer cet observation en regardant les valeurs maximum d'une précipitation de pluie ( http://thewatchers.adorraeli.com/2015/07/15/record-rainfall-freak-storm-dumps-102-mm-of-rain-in-1-hour-norway/ ). Les observation avec un précipitations > 102mm/h sont probablement des outliers.


In [7]:
df_train.corr()

,Id,minutes_past,radardist_km,Ref,Ref_5x5_10th,Ref_5x5_50th,Ref_5x5_90th,RefComposite,RefComposite_5x5_10th,RefComposite_5x5_50th,...,RhoHV_5x5_90th,Zdr,Zdr_5x5_10th,Zdr_5x5_50th,Zdr_5x5_90th,Kdp,Kdp_5x5_10th,Kdp_5x5_50th,Kdp_5x5_90th,Expected
Id,1.000000,0.000514,0.003874,-0.011175,-0.009607,-0.011636,-0.012517,-0.011757,-0.011157,-0.012030,...,0.009464,-0.002390,-0.015640,-0.011086,0.006011,0.000157,-0.013305,-0.008225,0.009491,-0.001001
minutes_past,0.000514,1.000000,-0.001427,-0.000239,0.003963,0.000792,-0.006013,-0.003078,0.002156,-0.002209,...,-0.000295,-0.000853,-0.005126,-0.003735,0.002630,-0.000242,-0.003394,-0.001035,0.002517,-0.000179
radardist_km,0.003874,-0.001427,1.000000,0.096163,0.142684,0.096597,0.033250,0.004996,0.027050,0.000654,...,0.249901,-0.152006,-0.152355,-0.287843,-0.205005,-0.006607,-0.233033,-0.092634,0.171412,0.106751
Ref,-0.011175,-0.000239,0.096163,1.000000,0.896970,0.958778,0.910152,0.931120,0.865410,0.914957,...,-0.064779,-0.061002,0.176643,-0.026197,-0.224567,0.002184,0.027295,0.077414,0.022467,-0.011127
Ref_5x5_10th,-0.009607,0.003963,0.142684,0.896970,1.000000,0.944505,0.856056,0.853667,0.933696,0.888494,...,-0.055624,-0.061566,0.192273,-0.057137,-0.238607,0.003335,0.022573,0.078655,0.034199,0.000150
Ref_5x5_50th,-0.011636,0.000792,0.096597,0.958778,0.944505,1.000000,0.939605,0.923970,0.912743,0.954114,...,-0.074839,-0.066665,0.191113,-0.027170,-0.245214,0.002444,0.032076,0.084926,0.022596,-0.008644
Ref_5x5_90th,-0.012517,-0.006013,0.033250,0.910152,0.856056,0.939605,1.000000,0.898427,0.846769,0.921637,...,-0.068607,-0.056707,0.166387,0.009601,-0.203275,0.001733,0.035098,0.087437,0.011711,-0.030052
RefComposite,-0.011757,-0.003078,0.004996,0.931120,0.853667,0.923970,0.898427,1.000000,0.911419,0.968428,...,-0.087785,-0.046229,0.190187,0.010437,-0.204699,0.001243,0.056856,0.081388,-0.002741,-0.019257
RefComposite_5x5_10th,-0.011157,0.002156,0.027050,0.865410,0.933696,0.912743,0.846769,0.911419,1.000000,0.947793,...,-0.089411,-0.048026,0.213321,-0.008281,-0.228475,0.002266,0.058498,0.082065,0.005047,-0.007461
RefComposite_5x5_50th,-0.012030,-0.002209,0.000654,0.914957,0.888494,0.954114,0.921637,0.968428,0.947793,1.000000,...,-0.098532,-0.049998,0.198200,0.011084,-0.217488,0.001375,0.061469,0.087105,-0.004102,-0.016464


Mis à part une lègère correlation entres les données des colonnes Ref, Ref_5x5_10th, Ref_5x5_50th, Ref_5x5_90th. les données ne semblent pas être corrélé deux a deux.

## Netoyyage des données

### Retirer les séquence où il y a un Ref nulle
https://www.kaggle.com/c/how-much-did-it-rain-ii/forums/t/16622/ignored-ids



In [10]:
df_train_clean = df_train.groupby('Id').apply( lambda gr: gr['Ref'].isnull().sum() > 0 );

### Agréger les données pour se débarasser de la dépendance à minutes_past

Le code qui vas suivre vas construire deux fichiers csv: clean_train.csv et clean_test.csv à partir des fichiers train.csv et test.csv et sur lesquelles vont êtres appliqué les opérations suivantes

 * Supprimer l'attribut minutes_past
 * Remplacer radardist_km par sample_weights
    - sample_weights représente un poid relatif à la véracité des valeurs de la ligne: nous somme partie du principe que plus les mesure ont été prise depuis une longue distance (radardist_km très grand) , plus les valeurs vont être imprécise.
 * Creation de la feature missing_values ( pourcentage des valeurs manquante sur chaque ligne )
 * Aggrégation des séquences par moyenne
    - En profiter pour créer des features max et median pour tout les attributs.
 * Remplir les NaN restant avec la médian de l'ensemble du jeu de données.
 * pour le fichier train.csv, supprimer les séquences avec un Expected > 102

In [ ]:
data_path = "data/"

input_files=[ "train.csv", "test.csv" ] # 13765201 lines

for input_file in input_files:
    print "Reading %s" % input_file
    time0 = dt.datetime.now()
    df = pd.read_csv(data_path+input_files)
    time1 = dt.datetime.now()
    print('Read in %i sec' % (time1-time0).seconds)

    print """Drop minutes_past columns"""
    df = df.drop('minutes_past', axis=1)

    print """Create sample_weights feature"""
    max_km = df["radardist_km"].max()
    df['sample_weights'] = df['radardist_km'].apply( lambda x:1-x/max_km )

    print """Drop radardist_km columns"""
    df = df.drop('radardist_km', axis=1)

    print """Create missing_values feature"""
    nb_extra_cols = 1
    if ( input_file == "train.csv"):
        nb_extra_cols = 2
    nb_columns = len(df.columns) - nb_extra_cols # without Expected and Id
    df['missing_values'] = df.isnull().sum(axis=1) / nb_columns

    grouped = df.groupby('Id')

    value_cols = [ 'Ref','Ref_5x5_10th','Ref_5x5_50th','Ref_5x5_90th',
    'RefComposite','RefComposite_5x5_10th','RefComposite_5x5_50th','RefComposite_5x5_90th',
    'RhoHV','RhoHV_5x5_10th','RhoHV_5x5_50th','RhoHV_5x5_90th',
    'Zdr','Zdr_5x5_10th','Zdr_5x5_50th','Zdr_5x5_90th',
    'Kdp','Kdp_5x5_10th','Kdp_5x5_50th','Kdp_5x5_90th' ]

    agg_cols = lambda col: {col+'_max' : np.max, col+'_median' : np.median}

    print """Agg by mean"""
    agg_df = grouped.agg(np.mean) 

    print """Create median, max aggregations""" 
    for col in value_cols:
        agg_df = pd.concat( [agg_df, grouped[col].agg(agg_cols(col))], axis=1 )


    print """Fill left NaN with median"""
    agg_df = agg_df.fillna(agg_df.median())


    if ( input_file == "train.csv"):
        print """Clean Expected column"""
        agg_df = agg_df[agg_df['Expected'] <= 102]


    clean_name = "clean_"+input_file
    print """Saving %s\n\n""" % clean_name
    agg_df.to_csv(data_path+clean_name)


## Filtrage des attributs

Sur les 61 attributs qu'on a crée, nous allons devoir choisir les plus pertinants.
Nous allons entrainer un premier modèle avec RandomForestRegressor et profiter de la proprieté feature_importances_ pour filtrer les attributs par degrée d'importance.

Sur les 61 attributs de départ, nous allons en garder 13 ( ceux qui me sembles les plus pertinents d'après la valeur renvoyé par feature_importances_ )

In [ ]:
print "RandomForestRegressor..."
clf = sklearn.ensemble.RandomForestRegressor(verbose=2, n_jobs=-1)
clf.fit(x_train, y_train)
clf.feature_importances_

## Entrainement des modèles
Nous avions choisis d'entrainer les données avec:
 * RandomForestRegressor
 * GradientBoostingRegressor
 * ExtraTreesRegressor
disponible dans la library sklearn.

Nous allons seulement sélectionner les 13 attributs qu'on a jugé pertinent lors de la précédente étape.

In [ ]:
cleantrain_file="data/clean_train.csv"

print "Lecture clean csv..."
time0 = dt.datetime.now()

cols = [ 'Id', 'Expected', 'missing_values',
'Ref_5x5_10th_max','Ref_5x5_50th_max','sample_weights',
'Ref_5x5_90th','RefComposite_5x5_90th','Ref_5x5_90th_max',
'RefComposite_5x5_90th_median','RefComposite_5x5_90th_max',
'Zdr_5x5_90th','Zdr_5x5_90th_max','Ref_5x5_90th_median','Zdr_5x5_90th_median']

df = pd.read_csv(cleantrain_file, usecols=cols)

time1 = dt.datetime.now()
print('lecture en: %i sec' % (time1-time0).seconds)
print "Nb lignes: %d ",  df.shape[0]

print "Prepare data, labels"
data = df.as_matrix()
label = df[['Expected']].as_matrix().ravel()
df = df.drop(['Expected', 'Id'], axis=1)

print "Prepare folds for cross validation"
x_train, x_test, y_train, y_test = cross_validation.train_test_split(data, label, test_size=0.4, random_state=23435)

print "RandomForestRegressor..."
clf = sklearn.ensemble.RandomForestRegressor(verbose=2, n_jobs=2)
clf.fit(x_train, y_train)
print mean_squared_error( clf.predict(x_test), y_test )
with open('models/RandomForestRegressor.pkl', 'wb') as fid:
    cpk.dump(clf, fid)

print "GradientBoostingRegressor..."
clf = sklearn.ensemble.GradientBoostingRegressor(verbose=2)
clf.fit(x_train, y_train)
print mean_squared_error( clf.predict(x_test), y_test )
with open('models/GradientBoostingRegressor.pkl', 'wb') as fid:
    cpk.dump(clf, fid)

print "ExtraTreesRegressor..."
clf = ExtraTreesRegressor(n_estimators=20, verbose=2, n_jobs=-1)
clf.fit(x_train, y_train)
print mean_squared_error( clf.predict(x_test), y_test )
with open('models/ExtraTreesRegressor.pkl', 'wb') as fid:
    cpk.dump(clf, fid)

## Création de la submission
Nous allons moyenner les résultats de prédiction des modèles qu'on a entrainé précédement avec ceux de la baseline (marshall-palmer) afin d'obtenir notre résultat final.

In [ ]:
model_path = "models/"
data_path = "data/"
cleantest_file="clean_test.csv"
models_files = ['GradientBoostingRegressor.pkl', 'RandomForestRegressor.pkl', 'ExtraTreesRegressor.pkl']


cols = [ 'Id', 'missing_values',
'Ref_5x5_10th_max','Ref_5x5_50th_max','sample_weights',
'Ref_5x5_90th','RefComposite_5x5_90th','Ref_5x5_90th_max',
'RefComposite_5x5_90th_median','RefComposite_5x5_90th_max',
'Zdr_5x5_90th','Zdr_5x5_90th_max','Ref_5x5_90th_median','Zdr_5x5_90th_median']

time0 = dt.datetime.now()
df = pd.read_csv(data_path+cleantest_file, usecols=cols)
time1 = dt.datetime.now()
print('lecture en: %i sec' % (time1-time0).seconds)
print "Nb lignes: %d ",  df.shape[0]

print "preparing data"
ids = df['Id']
df = df.drop(['Id'], axis=1)
data = df.as_matrix()


pred_exp = [pd.read_csv("data/marshall-palmer.csv")]

for model_file in models_files:
    with open(model_path+model_file, 'rb') as fid:
        print "Load classifier %s" % model_file
        clf = cpk.load(fid)

        print "Prediction..."
        pred = clf.predict(data)

        print "Saving csv..."
        df_pred = pd.DataFrame({'Expected': pred, 'Id': ids})
        pred_exp.append( pred )

        df_pred.to_csv(data_path+model_file+'_sol.csv', index=False)

sol = pd.DataFrame({'Id': ids, 'Expected': np.mean( pred_exp, axis=0) }) 
solution_file="solution.csv"
sol.to_csv(data_path+solution_file, header=True, cols=["Id","Expected"], index=False)

## Résultats
* Public  : 24.06898
* Private : 25.09764
    
Les résultats obtenu sont juste un peu mieux que la baseline. Une manière d'améliorer ses résultats est d'entrainer les modèles plus longtemps...